In [1]:
import numpy as np
from imshowtools import imshow
from PIL import Image
import os

from ipywidgets import interact
from ipywidgets.widgets import SelectionSlider, IntSlider
import re

In [2]:
@interact
def get_exp_name(exp=SelectionSlider(options=os.listdir('logs'), continuous_update=False)):

    exp_dir = f'logs/{exp}'
    versions = [int(d.replace('version_', '')) for d in os.listdir(exp_dir) if len(os.listdir(f'{exp_dir}/{d}/media')) > 0]

    @interact
    def get_config(version=SelectionSlider(options=versions, continuous_update=False)):
        version_dir = f'logs/VanillaVAE/version_{version}'
        config_file = os.path.join(version_dir, 'config.yaml')
        with open(config_file, 'r') as f:
            config = f.read()

        files = os.listdir(os.path.join(version_dir, 'media'))
        match = re.match('epoch_([0-9]*)', files[-1])
        max_epoch = int(match[1])

        @interact
        def get_imgs(epoch=IntSlider(value=0, min=0, max=max_epoch, continuous_update=False)):
            img_input = f'epoch_{epoch}_input.png'
            img_recons = f'epoch_{epoch}_recons.png'
            img_samples = f'epoch_{epoch}_samples.png'
            img_paths = [img_input, img_recons, img_samples]

            imgs = [np.asarray(Image.open(os.path.join(version_dir, 'media', p))) for p in img_paths]
            imshow(*imgs, title=img_paths, size=(30, 30))

            print('Configuration:\n')
            print(config)

        return None

interactive(children=(SelectionSlider(continuous_update=False, description='exp', options=('VanillaVAE',), val…